In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import glob
import os
from ahh import pre, vis
%matplotlib inline

In [ ]:
# warm water volume
df = pd.read_csv('data/csv/wwv_m3.csv')
df.index = pd.to_datetime(df['monthyear'], format='%Y%m')
df = df.drop('monthyear', axis=1)
da = pre.make_xr(df.wwv_m3.values, time=df.index.values)
da = da.groupby('time.month') - da.sel(time=slice('1981-01-01', '2010-12-31')).groupby('time.month').mean()
da = da.sel(time=slice('1982-01-01', '2017-12-31'))
da.name = 'wwv'
df = da.to_dataframe()
df = df.drop('month', axis=1)
df.index = pd.DatetimeIndex(da['time'].values)
df.index.name = 'time'
df.to_pickle('data/pkl/wwv_m3.pkl')

In [ ]:
ds = xr.open_dataset('data/nc/inino34_daily.nc').sel(time=slice('1982-01-01', '2017-12-31'))
ds = ds.dropna('time')
df = ds.to_dataframe()
df.columns = ['sst']
df.index = pd.DatetimeIndex(ds.time.values)
df.index.name = 'time'
df.to_pickle('data/pkl/sst_k.pkl')

In [ ]:
ds_list = []
nc_files = sorted(glob.glob('data/nc/uwnd.*.nc'))

for nc in nc_files:
    try:
        ds = xr.open_dataset(nc)
        ds = ds.sel(level=925, lat=slice(5, -5), lon=slice(120, 160))
        ds_list.append(ds)
    except:
        pass

ds_all = xr.concat(ds_list, 'time')

wnd_ds = ds_all

wnd_ds_avg = wnd_ds.mean('lat').mean('lon')

wnd_ds_anom = wnd_ds_avg.groupby('time.month') - \
               wnd_ds_avg.sel(time=slice('1982-01-01', '2010-12-31')).groupby('time.month').mean()

df = wnd_ds_anom['uwnd'].to_dataframe()
df = df.drop('month', axis=1)
df.to_pickle('data/pkl/wnd_ms.pkl')